## Домашнее задание "Функции потерь и оптимизация"

Реализовать самостоятельно логистическую регрессию

Обучить ее методом градиентного спуска

Методом nesterov momentum

Методом rmsprop



В качестве dataset’а взять Iris, оставив 2 класса:

Iris Versicolor

Iris Virginica

In [1]:
from sklearn import datasets
import numpy as np
from scipy.optimize import approx_fprime
import pandas as pd

In [366]:
# импортируем датасет, изучаем его

iris = datasets.load_iris()

In [367]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [20]:
iris.data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [21]:
tar = iris.target.reshape(150,1)

In [22]:
# добавляем в один массив входные данные и искомое значение

arr1 = np.append(iris.data, tar, axis =1)
arr1

array([[5.1, 3.5, 1.4, 0.2, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5. , 3.6, 1.4, 0.2, 0. ],
       [5.4, 3.9, 1.7, 0.4, 0. ],
       [4.6, 3.4, 1.4, 0.3, 0. ],
       [5. , 3.4, 1.5, 0.2, 0. ],
       [4.4, 2.9, 1.4, 0.2, 0. ],
       [4.9, 3.1, 1.5, 0.1, 0. ],
       [5.4, 3.7, 1.5, 0.2, 0. ],
       [4.8, 3.4, 1.6, 0.2, 0. ],
       [4.8, 3. , 1.4, 0.1, 0. ],
       [4.3, 3. , 1.1, 0.1, 0. ],
       [5.8, 4. , 1.2, 0.2, 0. ],
       [5.7, 4.4, 1.5, 0.4, 0. ],
       [5.4, 3.9, 1.3, 0.4, 0. ],
       [5.1, 3.5, 1.4, 0.3, 0. ],
       [5.7, 3.8, 1.7, 0.3, 0. ],
       [5.1, 3.8, 1.5, 0.3, 0. ],
       [5.4, 3.4, 1.7, 0.2, 0. ],
       [5.1, 3.7, 1.5, 0.4, 0. ],
       [4.6, 3.6, 1. , 0.2, 0. ],
       [5.1, 3.3, 1.7, 0.5, 0. ],
       [4.8, 3.4, 1.9, 0.2, 0. ],
       [5. , 3. , 1.6, 0.2, 0. ],
       [5. , 3.4, 1.6, 0.4, 0. ],
       [5.2, 3.5, 1.5, 0.2, 0. ],
       [5.2, 3.4, 1.4, 0.2, 0. ],
       [4.7, 3

In [23]:
iris.feature_names.append('Iris type')
df = pd.DataFrame(arr1, columns  = iris.feature_names)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Iris type
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [26]:
# оставляем только Iris Versicolor и Iris Virginica

df1 = df[df['Iris type'] != 2]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 5 columns):
sepal length (cm)    100 non-null float64
sepal width (cm)     100 non-null float64
petal length (cm)    100 non-null float64
petal width (cm)     100 non-null float64
Iris type            100 non-null float64
dtypes: float64(5)
memory usage: 4.7 KB


In [345]:
# Создаем массивы для обучения

x = np.array(X)
y = df1.iloc[:,4]

In [351]:
# Задаем функцию логистической регрессии

def f(w, x):
    pow = w[0] + w[1]*x[0] + w[2]*x[1] + w[3]*x[2] + w[4]*x[3]
    return 1/(1 + (np.e)**(-pow))

In [352]:
# Задаем функцию потерь

def f_min (w, x, y):
    pow = w[0] + w[1]*x[0] + w[2]*x[1] + w[3]*x[2] + w[4]*x[3]
    func = 1/(1 + (np.e)**(-pow))
    return -(y*np.log(func) + (1-y)*np.log(1-func))

In [353]:
# Задаем функцию расчета градиента

def avg_der(w, x, y):
    grad  = np.zeros(5)
    for i in range (len(x)-1):
        x2 = x[i,]
        y2 = y[i]
        grad = grad + approx_fprime(w, f_min, 1e-6, x2, y2)
    return grad/(len(x))

In [354]:
# Задаем функцию для расчета общей ошибки

def cost_fun(w, x, y):
    cost_fun = 0 
    for i in range (len(x)):
        x2 = x[i,]
        y2 = y[i]
        cost_fun = cost_fun + f_min(w, x2, y2)
    return cost_fun

In [356]:
# Градиентный спуск 

w = np.array([1,1,1,1,1])
cost = 0
learning_rate = 0.9

for _ in range(100):
    
    grad1 = avg_der(w, x, y)
    cost = cost_fun(w,x,y)
    w = np.array([w[0]-learning_rate*grad1[0],w[1]-learning_rate*grad1[1],w[2]-learning_rate*grad1[2],
                  w[3]-learning_rate*grad1[3],w[4]-learning_rate*grad1[4]])
    
    print(w, cost)

[ 0.55000882 -1.25265839 -0.54257208  0.34211323  0.88930073] 557.1010035089829
[0.98885318 1.3554317  0.67263531 2.21477854 1.47208808] 287.84790286570137
[ 0.53884667 -0.89728315 -0.86996665  1.5568624   1.36138013] 684.0039882207766
[ 0.63799794 -0.30520897 -0.60090117  1.9829195   1.49434532] 14.995780363461964
[ 0.37947807 -1.59390527 -1.47895389  1.59997757  1.42909183] 44.44898227638849
[ 0.81403174  0.98857337 -0.27503602  3.45214838  2.00532928] 224.12480832178255
[ 0.36405193 -1.26402881 -1.81756808  2.79427513  1.89463323] 518.0201901666219
[ 0.42132691 -0.93083336 -1.66134085  3.02191589  1.96540439] 7.516575659032882
[ 0.41991286 -0.93565918 -1.66746908  3.02590641  1.96755113] 0.7905864798023331
[ 0.41872842 -0.93930946 -1.67289774  3.03041188  1.96982728] 0.7818875015802081
[ 0.41770452 -0.94214262 -1.67783369  3.03526325  1.97218829] 0.7744466017124465
[ 0.4167937  -0.94440587 -1.68241866  3.04034219  1.97460268] 0.7676587055233136
[ 0.4159632  -0.94627008 -1.68675021  

In [355]:
# Nesterov momentum 

w = np.array([1,1,1,1,1])
cost = 0
gamma = 0.9
vt = np.zeros(5)

for _ in range(100):
    
    grad1 = avg_der(w, x, y)
    
    vt = np.array([gamma*vt[0] + learning_rate*grad1[0], gamma*vt[1] + learning_rate*grad1[1],
                        gamma*vt[2] + learning_rate*grad1[2], gamma*vt[3] + learning_rate*grad1[3],
                        gamma*vt[4] + learning_rate*grad1[4]])
        
    w = np.array([w[0]-vt[0],w[1]-vt[1],w[2]-vt[2],
                  w[3]-vt[3],w[4]-vt[4]])    
        
    
    cost = cost_fun(w,x,y)
    
    print(w, cost)

[0.95000098 0.74970462 0.8286031  0.92690147 0.98770008] 457.0856626191332
[0.85500863 0.27417148 0.50296755 0.7880228  0.96433187] 267.31941457006275
[ 0.71978399 -0.40278335  0.03937558  0.59034675  0.93107404] 38.6942243217032
[ 0.57907809 -1.1023428  -0.44642192  0.39823109  0.90162754] 216.37871765351105
[ 0.50053613 -1.44589052 -0.75050628  0.43095884  0.93914133] 354.0641212724189
[ 0.47878466 -1.46433239 -0.88865876  0.66908822  1.03783603] 322.5358234298297
[ 0.50810408 -1.19041365 -0.87757984  1.09190136  1.19153741] 141.4223103992532
[ 0.58015807 -0.67217224 -0.74105606  1.66718882  1.3904435 ] 6.082536266753037
[ 0.64281321 -0.21500367 -0.62658426  2.18612874  1.5706358 ] 66.81369402929239
[ 0.66312052  0.01620982 -0.64652323  2.59984109  1.72374474] 141.1619374846402
[ 0.6347544  -0.00918211 -0.82419074  2.90374389  1.84999385] 127.97503500578439
[ 0.56370761 -0.25981524 -1.139804    3.11036068  1.95232017] 47.993287024372044
[ 0.47043402 -0.63138662 -1.52288706  3.2524957

In [357]:
# RMSProp

w = np.array([1,1,1,1,1])
cost = 0
gamma = 0.9
grad_squared = np.zeros(5)
learning_rate = 0.1

for _ in range(100):
    
#  grad_squared = gamma * grads_squared + (1 - gamma) * dw * dw
#  w = w - (lr / np.sqrt(grad_squared)) * dw

    grad1 = avg_der(w, x, y)

    grad_squared = np.array([gamma*grad_squared[0] + (1-gamma)*grad1[0]**2, 
                             gamma*grad_squared[1] + (1-gamma)*grad1[1]**2,
                             gamma*grad_squared[2] + (1-gamma)*grad1[2]**2,
                             gamma*grad_squared[3] + (1-gamma)*grad1[3]**2,
                             gamma*grad_squared[4] + (1-gamma)*grad1[4]**2])


    w = np.array([w[0] - (learning_rate / np.sqrt(grad_squared[0] + 1e-8)) * grad1[0],
                  w[1] - (learning_rate / np.sqrt(grad_squared[1] + 1e-8)) * grad1[1],
                  w[2] - (learning_rate / np.sqrt(grad_squared[2] + 1e-8)) * grad1[2],
                  w[3] - (learning_rate / np.sqrt(grad_squared[3] + 1e-8)) * grad1[3],
                  w[4] - (learning_rate / np.sqrt(grad_squared[4] + 1e-8)) * grad1[4]])
                         
    cost = cost_fun(w,x,y)
                                
    print(w, cost)

[0.6837723  0.68377224 0.68377224 0.68377226 0.68377328] 380.95995131387565
[0.45442049 0.45441898 0.45441628 0.4544259  0.45443536] 253.54976551600558
[0.26323587 0.26323526 0.26317437 0.26345761 0.26374197] 150.23951823168252
[0.10101552 0.10129655 0.10035038 0.10535513 0.11110015] 80.11449269622474
[-0.00117984  0.00357464 -0.00711698  0.05568152  0.13683808] 62.703350860577956
[-0.02639778 -0.00864619 -0.04565988  0.16497584  0.35733492] 53.43097698532734
[-0.07515817 -0.04864228 -0.103507    0.21245899  0.48133706] 46.22951975977046
[-0.10585808 -0.06927852 -0.14440246  0.28661526  0.62057055] 40.405119466016046
[-0.14056605 -0.09532183 -0.18781925  0.34372093  0.73106583] 35.63816355507744
[-0.16951382 -0.11564087 -0.22535225  0.40420365  0.83710376] 31.658677584205464
[-0.19785123 -0.13594363 -0.26161304  0.45905584  0.93148067] 28.31183201013752
[-0.22383178 -0.15417793 -0.2951789   0.51242771  1.01959384] 25.464228817428
[-0.2485377  -0.17149159 -0.32707552  0.56301781  1.1011

In [266]:
from sklearn.model_selection import train_test_split

In [267]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [86]:
from sklearn.linear_model import LogisticRegression

In [87]:
#  Сравним с результатами, полученными с помощью sklearn

model = LogisticRegression()

In [358]:
model.fit(X_test,y_test)

D:\DS\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [359]:
model.coef_

array([[-0.30481354, -1.11324349,  1.62731963,  0.73381528]])

In [360]:
model.intercept_

array([-0.20401624])

In [361]:
model.score(X_test, y_test)

1.0

In [362]:
model.score(X_train, y_train)

1.0

In [363]:
# Для сравнения коэффициенты модели изменены на результаты RSMProp. Score не изменился. 

model.coef_ = np.array([[-0.9137235 , -1.90087989 , 3.23989037  , 4.49578444]])
model.intercept_ = np.array([-1.40024236 ])

In [364]:
model.score(X_test, y_test)

1.0

In [365]:
model.score(X_train, y_train)

1.0